In [1]:
# Importing the required libraries
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

In [2]:
os.getcwd()
os.chdir("E:\DataScience\Capstone_Proj")

In [3]:
# Importing the dataset
df = pd.read_csv("Purchase.csv")

In [4]:
df.head(10)
#df.dtypes

,QuoteNumber,Original_Quote_Date,QuoteConversion_Flag,Field6,Field7,Field8,Field9,Field10,Field11,Field12,...,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,1,2013-08-16,0,B,23,0.9403,0.0006,965,1.0200,N,...,9,9,-1,8,-1,18,-1,10,N,CA
1,2,2014-04-22,0,F,7,1.0006,0.0040,548,1.2433,N,...,10,10,-1,11,-1,17,-1,20,N,NJ
2,4,2014-08-25,0,F,7,1.0006,0.0040,548,1.2433,N,...,15,18,-1,21,-1,11,-1,8,N,NJ
3,6,2013-04-15,0,J,10,0.9769,0.0004,"1,165",1.2665,N,...,6,5,-1,10,-1,9,-1,21,N,TX
4,8,2014-01-25,0,E,23,0.9472,0.0006,"1,487",1.3045,N,...,18,22,-1,10,-1,11,-1,12,N,IL
5,12,2014-01-18,0,E,14,0.9472,0.0006,"1,487",1.3045,N,...,10,11,-1,23,-1,25,-1,14,N,IL
6,13,2013-11-01,0,J,23,0.9258,0.0004,"1,165",1.2665,N,...,18,22,-1,19,-1,13,-1,8,N,TX
7,14,2014-05-14,0,B,25,0.9153,0.0007,935,1.0200,N,...,6,5,-1,3,-1,12,-1,13,N,CA
8,18,2013-06-19,0,J,10,0.9691,0.0004,"1,165",1.2665,N,...,2,1,-1,3,-1,2,-1,13,N,TX
9,19,2013-06-18,0,F,11,0.9919,0.0038,564,1.1886,N,...,10,11,-1,9,-1,22,-1,18,N,NJ


In [5]:
# Extracting day of week, month and year from 'Original_Quote_Date' using datetime function from pandas
df['Date'] = pd.to_datetime(pd.Series(df['Original_Quote_Date']))
df = df.drop(["Original_Quote_Date","QuoteNumber"], axis=1)
df['weekday'] = df['Date'].dt.dayofweek
df['weekyear'] = df['Date'].dt.weekofyear
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [7]:
#df.isnull().count()
#df.isna().count()
#df.isnull().sum()
#df = df.drop(["weekyear","Year"], axis=1)
#df.describe()
#sum(df.isnull().values.ravel())

In [8]:
df = df.drop(["Date"], axis=1)

In [9]:
df.isnull().values.ravel().sum()

326623

In [10]:
df = df.fillna(-99) # Filling missing values with some random number

In [11]:
# Converting the object columns to numeric using LableEncoder
for f in df.columns:
    if df[f].dtype == 'object':
        #print(f)
        lb1 = LabelEncoder()
        lb1.fit(list(df[f].values))
        df[f] = lb1.transform(list(df[f].values))

In [12]:
# Splitting the dataset into the Training and Test set
from sklearn.model_selection import train_test_split
X = df.drop(['QuoteConversion_Flag'], axis=1)
y = df['QuoteConversion_Flag']
#y.dtype

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

(208602, 300)
(208602,)


In [13]:
#X_train

# Xgboost     https://xgboost.readthedocs.io/en/latest/parameter.html

In [14]:
# Fitting XGBoost to the Training set
clf = xgb.XGBClassifier(objective= "binary:logistic",
                       eta = 0.1,
                       n_estimators = 100,
                       verbose =1,
                       silent = False)

xbg_mobel = clf.fit(X_train, y_train, eval_metric="error")

In [15]:
clf

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
       subsample=1, verbose=1)

In [22]:
# Predicting the test results and Making the confusion matrix
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

pred = xbg_mobel.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
pred[1]

0

In [24]:
#predTrain = xbg_mobel.predict(X_train)

In [25]:
probs = xbg_mobel.predict_proba(X_test)

In [26]:
print(classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.92      0.99      0.95     42231
          1       0.94      0.61      0.74      9920

avg / total       0.92      0.92      0.91     52151



In [21]:
# print(classification_report(y_train,predTrain)) ## no need

In [27]:
print(accuracy_score(pred,y_test))

0.9193304059366072


In [28]:
# Trying to improve the accuracy by changing max_depth parameters
clf1 = xgb.XGBClassifier(objective= "binary:logistic",
                       eta = 0.1,
                       n_estimators = 100,max_depth=10,
                       verbose =10,
                       silent = 0)

In [30]:
xbg_mobel = clf1.fit(X_train, y_train, eval_metric="error")

In [31]:
pred = xbg_mobel.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [32]:
probs = xbg_mobel.predict_proba(X_test)

In [33]:
print(classification_report(y_test,pred))

             precision    recall  f1-score   support

          0       0.93      0.98      0.96     42231
          1       0.90      0.69      0.78      9920

avg / total       0.92      0.93      0.92     52151



In [35]:
print(accuracy_score(pred,y_test))

0.9259458112020863


In [36]:
# Applying KFold Cross Validation
from sklearn.model_selection import StratifiedKFold,cross_val_score

In [37]:
kfold = StratifiedKFold(n_splits=3, random_state=7)
result = cross_val_score(clf,X_train,y_train,cv=kfold)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [38]:
result
## we wiil get each fold acc

array([0.92153592, 0.9208301 , 0.91867171])

In [39]:
result.std()

0.001218400697914653

In [40]:
result.mean()

0.9203459081686306